<a href="https://colab.research.google.com/github/michael-wettach/pythonsamples/blob/main/Python_5_Systemfunktionen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Module zur Interaktion mit dem Betriebssystem</h1>

Frühere Versionen von Python nutzten dafür in erster Linie die Module sys und os. Aktuelle Versionen setzen eher auf das Modul subprocess; alle 3 haben in gewissen Bereichen ihre Berechtigung und sollen hier vorgestellt werden.

<h2>Modul sys</h2>

Das Modul sys stellt Informationen über die Laufzeitumgebung des aktuellen Python-Programms zur Verfügung. 

In [4]:
import sys
# Informationen über die Laufzeitumgebung
print(sys.version)              # Gibt die aktuelle Python-Version aus
print(sys.version_info)         # Versionsinfos als Objekt mit Attributen
print(sys.platform)             # Betriebssystem, auf dem Python läuft
print(sys.executable)           # Pfad zu Python
print(sys.path)                 # Aktueller Inhalt der path-Variable
print(sys.getrecursionlimit())  # Maximale Rekursionstiefe
sys.setrecursionlimit(500)      # Kann man auch verändern
print(sys.getrecursionlimit())  # Sollte jetzt 500 sein

3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]
sys.version_info(major=3, minor=7, micro=10, releaselevel='final', serial=0)
linux
/usr/bin/python3
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']
500
500


In [9]:
# Das Modul sys enthält 3 Filehandles sys.stdin, sys.stdout, sys.stderr
# Diese gibt es auch auf Betriebssystem-Ebene in Unix und Windows. 
# Die kann man zur Eingabe und Ausgabe nutzen oder auch umleiten.

print("Textausgabe auf stdout", file=sys.stdout)
sys.stdout.write("So kann man auch Text ausgeben...\n")

# Jetzt leiten wir das Filehandle stdout in eine Datei um
save_stdout = sys.stdout     # für spätere Wiederherstellung

with open("test.txt","w") as fh:
    sys.stdout = fh
    print("Diese Ausgabe wandert in test.txt")

# Hier wird die Umleitung zurück genommen
sys.stdout = save_stdout
print("Alles wieder normal")

Textausgabe auf stdout
So kann man auch Text ausgeben...
Alles wieder normal


<h2>Modul os</h2>

Das Modul os stellt Funktionen für die Interaktion zwischen Python und dem Betriebssystem zur Verfügung. Generell sollte das Modul unabhängig vom Betriebssystem funktionieren, es gibt aber im Modul os Funktionen, die nur auf Linux oder nur auf Windows bereitgestellt werden oder sich geringfügig unterschiedlich verhalten. Das gilt natürlich erst recht für die Betriebssystem-Befehle, die über os-Funktionen aufgerufen werden. Wenn ein Python-Modul auf mehreren Plattformen laufen soll, muss man die Plattform abfragen.

In [30]:
# Betriebssystem-Aufruf ohne Python
!ls -l

total 8
drwxr-xr-x 1 root root 4096 Jun  1 13:40 sample_data
-rw-r--r-- 1 root root   34 Jun 12 15:23 test.txt


In [13]:
import sys
import os
if sys.platform == "linux":
    output = os.popen("ls -l")  # führt einen Befehl aus und öffnet ein Handle auf die Ausgabe
    dir = output.readlines()    # readlines() liest aus dem Handle in eine Liste
print(*dir)

total 8
 drwxr-xr-x 1 root root 4096 Jun  1 13:40 sample_data
 -rw-r--r-- 1 root root   34 Jun 12 15:23 test.txt



Python bietet auch die Möglichkeit, einen selbständigen Unterprozess zu starten, der nicht zum Hauptprozess zurückkehrt.<br/> Dafür gibt es die Exec* Funktionen, die ich hier aber nicht weiter erläutere (siehe dazu https://www.python-kurs.eu/forking.php).

<h2>Modul subprocess</h2>

In neueren Python-Versionen wird für Betriebssystem-Aufrufe das Modul subprocess empfohlen. Siehe dazu
* https://docs.python.org/3/library/subprocess.html
* https://www.digitalocean.com/community/tutorials/how-to-use-subprocess-to-run-external-programs-in-python-3-de 

In [24]:
import subprocess
# Die Funktion check_output erspart das Auslesen des Filehandles 
if sys.platform == "linux":
    output = subprocess.check_output(["ls", "-l"])
# Die Ausgabe ist aber im Format byte Array, daher decode()
print(output.decode("utf-8"))    

total 8
drwxr-xr-x 1 root root 4096 Jun  1 13:40 sample_data
-rw-r--r-- 1 root root   34 Jun 12 15:23 test.txt



In [28]:
# Auf Betriebssystem-Ebene möchten wir evtl. auch pipes nutzen
!dmesg | grep "VFS"

[    0.866703] VFS: Disk quotas dquot_6.6.0
[    0.867533] VFS: Dquot-cache hash table entries: 512 (order 0, 4096 bytes)
[    1.601612] VFS: Mounted root (ext2 filesystem) readonly on device 253:0.


In [42]:
# Das geht natürlich auch im Python Modul subprocess
p1 = subprocess.Popen(["dmesg"], stdout=subprocess.PIPE)                          # p1 leitet die Ausgabe in die pipe
p2 = subprocess.Popen(["grep", "VFS"], stdin=p1.stdout, stdout=subprocess.PIPE)   # holt die Ausgabe aus p1 
p1.stdout.close()                                       # wichtig, damit p1 SIGPIPE bekommt, falls p2 früher endet.

# Hinweis: Statt .stdin.write, .stdout.read or .stderr.read soll communicate() benutzt werden
# um deadlocks zu vermeiden falls andere OS pipe buffers den Kind-Prozess füllen bzw. blockieren.
output, retcode = p2.communicate()
print(output.decode("utf-8"))                                                     # Auch dies ist ein Byte Array
print("Returncode: ", retcode)                                                    # retcode < 0 ist ein Fehler.
p2.stdout.close()

[    0.866703] VFS: Disk quotas dquot_6.6.0
[    0.867533] VFS: Dquot-cache hash table entries: 512 (order 0, 4096 bytes)
[    1.601612] VFS: Mounted root (ext2 filesystem) readonly on device 253:0.

Returncode:  None
